In [1]:
import pandas as pd
import numpy as np
import itertools
pd.options.mode.chained_assignment = None

#read in csv file
df = pd.read_csv('projectData2.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9762 entries, 0 to 9761
Data columns (total 7 columns):
Unnamed: 0    9762 non-null int64
coID          9762 non-null int64
sales         9762 non-null float64
inventory     9762 non-null float64
strata_1      9762 non-null int64
strata_2      9762 non-null int64
strata_3      9762 non-null int64
dtypes: float64(2), int64(5)
memory usage: 533.9 KB


In [2]:
pctl_freq = df.groupby(['strata_1'])['sales'].count()
print pctl_freq

strata_1
1     4285
2     1988
3     1308
4      919
5      554
6      314
7      201
8      125
9       57
10      11
Name: sales, dtype: int64


In [3]:
pctl_freq = df.groupby(['strata_2'])['sales'].count()
print pctl_freq

strata_2
1     2560
3     1725
4     1988
5      707
6     1099
7      740
8      410
9      419
10     114
Name: sales, dtype: int64


In [4]:
pctl_freq = df.groupby(['strata_3'])['sales'].count()
print pctl_freq

strata_3
1    4208
2    3435
3    1212
4     529
5     280
6      85
7      11
8       2
Name: sales, dtype: int64


In [5]:
df2 = pd.read_csv('projectData_strata.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9762 entries, 0 to 9761
Data columns (total 7 columns):
coID              9762 non-null int64
sales             9762 non-null float64
inventory         9762 non-null float64
cum_inventory     9762 non-null float64
Cum_Stratum       9762 non-null int64
E_Stratum_3       9762 non-null int64
Cum_fy_Stratum    9762 non-null int64
dtypes: float64(3), int64(4)
memory usage: 533.9 KB


In [22]:
pctl_freq2 = df2.groupby(['Cum_Stratum'])['inventory'].count()
print pctl_freq2

Cum_Stratum
1     5728
2     1680
3     1002
4      600
5      361
6      202
7      122
8       51
9       14
10       2
Name: inventory, dtype: int64


In [31]:
def neymanAlloc(df, strata_field, sample_size):
    neyman_alloc = pd.DataFrame()
    neyman_alloc['stratum'] = df.groupby([strata_field], as_index = False).count()[strata_field]
    neyman_alloc['N_h'] = df.groupby([strata_field], as_index = False).count()['coID']
    neyman_alloc['S_h'] = df.groupby([strata_field], as_index = False).std()['inventory'])
    neyman_alloc['N_h*S_h'] = df.groupby([strata_field], as_index = False).std()['inventory'] * neyman_alloc['N_h']
    neyman_alloc['N_h*S_h/sum(N_h*S_h)'] = neyman_alloc['N_h*S_h'] / neyman_alloc['N_h*S_h'].sum()
    neyman_alloc['samp*N_h*S_h/sum(N_h*S_h)'] = sample_size * neyman_alloc['N_h*S_h/sum(N_h*S_h)']
    neyman_alloc['sample size'] = neyman_alloc['samp*N_h*S_h/sum(N_h*S_h)'].round().astype('int')
    
    print neyman_alloc
    return neyman_alloc['N_h'], neyman_alloc['sample size']

In [20]:
strata_1_Nh, strata_1_neyman = neymanAlloc(df2, 'Cum_Stratum', 500)

   stratum   N_h           S_h       N_h*S_h  N_h*S_h/sum(N_h*S_h)  \
0        1  5728  1.867460e+04  1.069681e+08              0.268932   
1        2  1680  1.759620e+04  2.956162e+07              0.074322   
2        3  1002  2.293439e+04  2.298025e+07              0.057775   
3        4   600  4.032405e+04  2.419443e+07              0.060828   
4        5   361  7.946726e+04  2.868768e+07              0.072125   
5        6   202  1.610430e+05  3.253069e+07              0.081787   
6        7   122  2.699542e+05  3.293441e+07              0.082802   
7        8    51  9.694394e+05  4.944141e+07              0.124302   
8        9    14  4.658438e+06  6.521813e+07              0.163967   
9       10     2  2.617081e+06  5.234163e+06              0.013159   

   samp*N_h*S_h/sum(N_h*S_h)  sample size  
0                 134.466233          134  
1                  37.160968           37  
2                  28.887744           29  
3                  30.414048           30  
4        

In [32]:
pctl_freq2 = df2.groupby(['Cum_fy_Stratum'])['inventory'].count()
print pctl_freq2

Cum_fy_Stratum
1    4215
2    1513
3     963
4     717
5     556
6     781
7     471
8     357
9     189
Name: inventory, dtype: int64


In [33]:
strata_1_Nh, strata_1_neyman = neymanAlloc(df2, 'Cum_fy_Stratum', 500)

UnsupportedFunctionCall: numpy operations are not valid with groupby. Use .groupby(...).std() instead

In [29]:
df3 = df2[df2['Cum_fy_Stratum'] < 9]
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9573 entries, 0 to 9572
Data columns (total 7 columns):
coID              9573 non-null int64
sales             9573 non-null float64
inventory         9573 non-null float64
cum_inventory     9573 non-null float64
Cum_Stratum       9573 non-null int64
E_Stratum_3       9573 non-null int64
Cum_fy_Stratum    9573 non-null int64
dtypes: float64(3), int64(4)
memory usage: 598.3 KB


In [30]:
strata_1_Nh, strata_1_neyman = neymanAlloc(df3, 'Cum_fy_Stratum', 311)

   stratum   N_h            S_h       N_h*S_h  N_h*S_h/sum(N_h*S_h)  \
0        1  4215    6057.624256  2.553289e+07              0.133827   
1        2  1513   14196.215417  2.147887e+07              0.112578   
2        3   963    6447.506755  6.208949e+06              0.032543   
3        4   717   10347.934568  7.419469e+06              0.038888   
4        5   556   10418.146185  5.792489e+06              0.030360   
5        6   781   36691.487369  2.865605e+07              0.150196   
6        7   471   55035.747971  2.592184e+07              0.135865   
7        8   357  195462.254176  6.978002e+07              0.365741   

   samp*N_h*S_h/sum(N_h*S_h)  sample size  
0                  41.620124           42  
1                  35.011843           35  
2                  10.120956           10  
3                  12.094176           12  
4                   9.442102            9  
5                  46.711069           47  
6                  42.254137           42  
7       

In [9]:
pctl_freq2 = df2.groupby(['E_Stratum_3'])['sales'].count()
print pctl_freq2

E_Stratum_3
1    3368
2    2473
3    1535
4     842
5     556
6     364
7     241
8     381
9       2
Name: sales, dtype: int64


In [14]:
strata_1_Nh, strata_1_neyman = neymanAlloc(df2, 'E_Stratum_3', 500)

   stratum   N_h       N_h*S_h  N_h*S_h/sum(N_h*S_h)  \
0        1  3368  9.434446e+07              0.077682   
1        2  2473  1.485278e+08              0.122296   
2        3  1535  1.053583e+08              0.086751   
3        4   842  8.007205e+07              0.065930   
4        5   556  7.641923e+07              0.062923   
5        6   364  5.952732e+07              0.049014   
6        7   241  4.543605e+07              0.037411   
7        8   381  5.544115e+08              0.456495   
8        9     2  5.040022e+07              0.041499   

   samp*N_h*S_h/sum(N_h*S_h)  sample size  
0                  38.840961           39  
1                  61.147875           61  
2                  43.375284           43  
3                  32.965111           33  
4                  31.461269           31  
5                  24.506986           25  
6                  18.705709           19  
7                 228.247381          228  
8                  20.749423           21  